# Introduction/Business Problem
The COVID-19 pandemic has taken an unprecedented toll globally, already affecting over 2M people, taking over 200K lives and, according to WEF, will likely cost the world 2 trillion USD in economic losses.

Not all cities are affected by the virus the same way. We want to see how poverty incidence, a metric used to measured the level of poverty in a geographic area, is a predictor for severity of problem as measured by cases per capita.

# Data Section
For this exercise, we will be using the following datasets:

1. **Case Information dataset**
    
    **Source:** Philippine Department of Health. See https://www.doh.gov.ph/2019-nCoV
    
    **Description:** List of confirmed COVID-19 cases from the DOH Epidemiological Bureau.

    **Dataset fields:**
	* CaseCode : Random code assigned for labelling cases
	* Age : Age
	* AgeGroup : Five-year age group
	* Sex : Sex
	* DateRepConf : Date publicly announced as confirmed case
	* DateRecover : Date recovered
	* DateDied : Date died
	* RemovalType : Type of removal (recovery or death)
	* DateRepRem : Date publicly announced as removed
	* Admitted : Binary variable indicating patient has been admitted to hospital
	* RegionRes : Region of residence
	* ProvCityRes : Province of residence
	* RegionPSGC : Philippine Standard Geographic Code of Region of Residence
	* ProvPSGC : Philippine Standard Geographic Code of Province of Residence
	* MunCityPSGC : Philippine Standard Geographic Code of Municipality or City of Residence
	* HealthStatus : Known current health status of patient (asymptomatic, mild, severe, critical, died, recovered)
	* Quarantined : Ever been home quarantined, not necessarily currently in home quarantine


2. **Municipal and City Level Poverty Estimates**

    **Source:** The Philippine Statistics Authority (PSA). See https://psa.gov.ph/content/psa-releases-2015-municipal-and-city-level-poverty-estimates

    **Description:** This is a set of estimates using the small area estimation (SAE) technique. See https://psa.gov.ph/sites/default/files/Technical%20Notes%20on%202015%20SAE.pdf

    **Dataset fields:**
    * PSGC
    * Region/Province
    * Municipality/City
    * Poverty Incidence


2. **Philippine Standard Geographic Code (PSGC)**

    **Source:** The Philippine Statistics Authority (PSA). See https://psa.gov.ph/content/psa-releases-2015-municipal-and-city-level-poverty-estimates

    **Description:** The PSGC is a systematic classification and coding of geographic areas in the Philippines. It is based on the four (4) well-established hierarchical levels of geographical-political subdivisions of the country, namely, the administrative region, the province, the municipality/city, and the barangay.

    **Dataset fields:**
    
    * Column "Code": PSGC Code
    * Column "Income Classification"
    * Column "Class" (Average Annual Income)
        * Provinces:
            * 1st	P 450M or more
            * 2nd	P 360M or more but less than P 450M
            * 3rd	P 270M or more but less than P 360M
            * 4th	P 180M or more but less than P 270M
            * 5th	P 90M or more but less than P 180M
            * 6th	Below P 90M
        * Cities
            * 1st	P 400M or more
            * 2nd	P 320M or more but less than P 400M
            * 3rd	P 240M or more but less than P 320M
            * 4th	P 160M or more but less than P 240M
            * 5th	P 80M or more but less than P 160M
            * 6th	Below P 80M
        * Municipalities
            * 1st	P 55M or more
            * 2nd	P 45M or more but less than P 55M
            * 3rd	P 35M or more but less than P 45M
            * 4th	P 25M or more but less than P 35M
            * 5th	P 15M or more but less than P 25M
            * 6th	Below P 15M		
    * Column "Urban / Rural"
	* Column "Population"	
    


2. Geojson dataset of cities and municipalities in the Philippines. We will use this dataset to provide the mapping boundaries of the different cities and municipalities in the Philippines.
	* ID_0 : Unique ID 0
	* ISO : ISO Country Code
	* NAME_0 : Country Name
	* NAME_2 : Municipality or City Name
	* PROVINCE : Province Name
	* REGION : Regiona Name
	* geometry : Polygon, coordinates

# Methodology
section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

We will use the CRISP-DM (Cross-Industry Process for Data Mining) methodology. The CRISP-DM methodology is well-proven methodology in data science. CRISP-DM loosely and iteratively follows six major phases:

1. Business Understanding
2. Data Understanding
3. Data Preparation
4. Modeling
5. Evaluation
6. Deployment

As we have have covered #1 and #2 previously, we will continue with Data Preparation.

# Results

# Discussion

# Conclusion

## Data Preparation

### Loading Data

1. We start by importing all necessary libraries and installing all dependencies for the project.

## plan

- [Setup](#setup)
    - import libraries
- [Prepare data](#data_prep)
    - DOH data
        - explore
        - clean
        - select
    - Poverty data
        - explore
        - clean
        - select
    - City index data
        - explore
        - clean
        - select
    - City Latlong
        - load json
        - explore
        - clean
        - select
    - Merge data
        - calculate case/pop as predicted variable
        - calculate distance from manila as additional feature
- Predict
- Visualize

<a name="setup"></a>
### Setup

<a name="import_lib"></a>
#### Import libraries

In [1]:
import numpy as np
import pandas as pd
import json5 as json # library to handle JSON files
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
import seaborn as sns
import xlrd
import shapely
import fiona
import pyproj
import geopandas as gpd

#prep
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MaxAbsScaler, QuantileTransformer

#models
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, LinearRegression, Ridge, RidgeCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

#validation libraries
from sklearn.model_selection import KFold, StratifiedKFold
from IPython.display import display
from sklearn import metrics



print('Libraries imported.')

Libraries imported.


<a name='data_prep'></a>
### Data Preparation

#### DOH Data

1. We define the columns and their proper data types.

In [2]:
#define column types
col_types = {'Age' : 'float64', 
             'AgeGroup' : 'category', 
             'Sex' : 'category', 
             'RemovalType' : 'category', 
             'Admitted' : 'category', 
             'RegionRes' : 'category', 
             'ProvRes' : 'category', 
             'CityMunRes' : 'category', 
             'CityMuniPSGC' : 'category',
             'BarangayRes' : 'category',
             'BarangayPSGC' : 'category',
             'HealthStatus' : 'category', 
             'Quarantined' : 'category', 
             'Pregnanttab' : 'category', 
             'ValidationStatus' : 'category'}
col_date = [4, 5, 6, 7, 8, 19]

3. We read the csv and load it into a data frame.

In [3]:
df_cases_raw = pd.read_csv("DOH COVID Data Drop_ 20201013 - 04 Case Information.csv", dtype = col_types, parse_dates = col_date)

##### Exploratory Data Analysis
1. We do a quick check on the dataframe

In [4]:
#check our dataframe
df_cases_raw.info()
df_cases_raw.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344713 entries, 0 to 344712
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   CaseCode           344713 non-null  object        
 1   Age                342504 non-null  float64       
 2   AgeGroup           342504 non-null  category      
 3   Sex                344713 non-null  category      
 4   DateSpecimen       296568 non-null  datetime64[ns]
 5   DateResultRelease  298652 non-null  datetime64[ns]
 6   DateRepConf        344713 non-null  datetime64[ns]
 7   DateDied           6334 non-null    datetime64[ns]
 8   DateRecover        74015 non-null   datetime64[ns]
 9   RemovalType        299755 non-null  category      
 10  Admitted           344559 non-null  category      
 11  RegionRes          340934 non-null  category      
 12  ProvRes            319787 non-null  category      
 13  CityMunRes         309212 non-null  category

,CaseCode,Age,AgeGroup,Sex,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,RemovalType,...,ProvRes,CityMunRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus
0,C900280,35.0,35 to 39,MALE,2020-07-03,2020-07-29,2020-08-05,NaT,NaT,RECOVERED,...,CAVITE,IMUS CITY,PH042109000,NaN,NaN,RECOVERED,NO,NaT,NaN,"Removal Type is ""Recovered"", but no Recovered ..."
1,C778257,32.0,30 to 34,FEMALE,2020-07-15,2020-07-17,2020-07-20,NaT,NaT,RECOVERED,...,COTABATO CITY (NOT A PROVINCE),NaN,NaN,NaN,NaN,RECOVERED,NO,NaT,NO,"Removal Type is ""Recovered"", but no Recovered ..."
2,C778303,54.0,50 to 54,MALE,2020-07-24,2020-07-29,2020-08-02,NaT,NaT,RECOVERED,...,NCR,CITY OF MANILA,PH133901000,NaN,NaN,RECOVERED,NO,NaT,NaN,"Health Status is ""Recovered"", but no Date Reco..."
3,C928860,33.0,30 to 34,MALE,NaT,NaT,2020-06-01,NaT,NaT,RECOVERED,...,NaN,NaN,NaN,NaN,NaN,RECOVERED,YES,NaT,NaN,"Health Status is ""Recovered"", but no Date Reco..."
4,C620602,1.0,0 to 4,FEMALE,2020-07-13,2020-07-16,2020-07-30,NaT,2020-07-30,RECOVERED,...,NCR,NaN,NaN,NaN,NaN,RECOVERED,NO,NaT,NO,NaN


In [5]:
#determine unique values per column
df_cases_raw.nunique(axis=0)

CaseCode             344713
Age                     104
AgeGroup                 17
Sex                       2
DateSpecimen            225
DateResultRelease       224
DateRepConf             225
DateDied                217
DateRecover             223
RemovalType               2
Admitted                  2
RegionRes                18
ProvRes                  84
CityMunRes             1360
CityMuniPSGC           1561
BarangayRes            8784
BarangayPSGC          12338
HealthStatus              6
Quarantined               2
DateOnset               227
Pregnanttab               2
ValidationStatus        502
dtype: int64

2. Since our study wants to see if certain city statistic predicts cases, we will limit our variables to cases and cities, i.e CityMuniPSGC

In [6]:
df_cases = df_cases_raw

In [7]:
df_cases = df_cases[['CaseCode','DateRepConf','CityMuniPSGC','CityMunRes']]

3. We drop the rows where we can't identify the cities

In [8]:
#drop Nan cities
df_cases = df_cases.loc[-df_cases.CityMuniPSGC.isna(),:]

4. For consistency, we rename CityMuniPSGC column to PSGC, which stands for Philippine Standard Geographic Code.

In [9]:
df_cases = df_cases.rename(columns={"CityMuniPSGC": "PSGC"})

5. Then we group the dataframe into cities are represented by PSGC and count the cases.

In [10]:
df_cases_by_city = df_cases.groupby(['PSGC','CityMunRes'])['CaseCode'].count().reset_index()

6. Then we transform PSGC to standard nine-digit code for consistency

In [11]:
df_cases_by_city.PSGC = df_cases_by_city.PSGC.str.replace('PH','')
df_cases_by_city.rename(columns={"CaseCode": "Cases"}, inplace = True)
df_cases_by_city.rename(columns={"CityMunRes": "Name"}, inplace = True)
df_cases_by_city.sort_values('PSGC', inplace = True)
df_cases_by_city.set_index('PSGC', inplace = True)
#check
df_cases_by_city.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2122960 entries, 012802000 to 175917000
Data columns (total 2 columns):
 #   Column  Dtype   
---  ------  -----   
 0   Name    category
 1   Cases   int64   
dtypes: category(1), int64(1)
memory usage: 36.5+ MB


In [12]:
df_cases_by_city = df_cases_by_city.loc[df_cases_by_city.Cases > 0,:].copy()

In [13]:
df_cases_by_city.head()

,Name,Cases
PSGC,,
012802000,BACARRA,4
012803000,BADOC,8
012804000,BANGUI,7
012805000,CITY OF BATAC,16
012806000,BURGOS,4


### Poverty data

In [14]:
#read city stats
df_poverty = pd.read_csv('City and Municipal-level Small Area Poverty Estimates_ 2009, 2012 and 2015_0.csv',
                  skiprows = [0,1,2,3,4],
                  encoding = "ISO-8859-1",
                  names = ['PSGC_ID','Poverty Incidence'],
                  usecols = [0,5],
                  dtype = {'PSGC_ID':'object','Poverty Incidence':np.float16})

In [15]:
#drop NaN indexes
df_poverty = df_poverty.loc[df_poverty.index.dropna()]

In [16]:
#drop other invalid rows
df_poverty = df_poverty.loc[-df_poverty.PSGC_ID.isna()]

In [17]:
df_poverty['PSGC'] = df_poverty.PSGC_ID.str.zfill(6)
df_poverty['PSGC'] = df_poverty.PSGC.str.ljust(9, '0')
df_poverty.sort_values('PSGC', inplace = True)
df_poverty.drop('PSGC_ID', axis =1, inplace = True)
df_poverty.set_index('PSGC', inplace = True)
df_poverty.sort_index(inplace = True)
#check
df_poverty.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1644 entries, 012801000 to 175917000
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Poverty Incidence  1644 non-null   float16
dtypes: float16(1)
memory usage: 16.1+ KB


### City index data

In [18]:
#load city data
df_cityindex = pd.read_excel('PSGC 2Q 2020 Publication.xlsx',
                    sheet_name = 'PSGC',
                    usecols = [i for i in range(7)],
                    names = ['PSGC',
                             'Name',
                             'Geographic Level',
                             'City Class',
                             'Income Classification',
                             'Urban Rural',
                             'Population'])

In [19]:
#filter to only include cities, municipalities and sub municipalities
df_cityindex = df_cityindex[df_cityindex['Geographic Level'].isin(['City','Mun','SubMun'])]

In [20]:
#replace NaNs with UNK
df_cityindex = df_cityindex.fillna('UNK')

In [21]:
df_cityindex.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1648 entries, 2 to 43788
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   PSGC                   1648 non-null   int64 
 1   Name                   1648 non-null   object
 2   Geographic Level       1648 non-null   object
 3   City Class             1648 non-null   object
 4   Income Classification  1648 non-null   object
 5   Urban Rural            1648 non-null   object
 6   Population             1648 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 103.0+ KB


In [22]:
df_cityindex.nunique()

PSGC                     1648
Name                     1445
Geographic Level            3
City Class                  4
Income Classification      17
Urban Rural                 1
Population               1636
dtype: int64

In [23]:
df_cityindex['Geographic Level'].unique()

df_cityindex['City Class'].unique()

df_cityindex['Income Classification'].unique()

df_cityindex['Urban Rural'].unique()

df_cityindex['Income Classification'] = df_cityindex['Income Classification'].str.replace(r'\*.*','')

df_cityindex['Income Classification'] = df_cityindex['Income Classification'].str.replace(r' .*','')

df_cityindex['Income Classification'] = df_cityindex['Income Classification'].str.replace('-','UNK')

df_cityindex.drop('Urban Rural', axis =1, inplace = True)

df_cityindex.PSGC = df_cityindex.PSGC.astype('str')

df_cityindex.PSGC = df_cityindex.PSGC.str.zfill(9)

df_cityindex = df_cityindex.astype(dtype = {'PSGC':'category', 'Name':'category',
                    'Geographic Level':'category',
                    'City Class':'category',
                    'Income Classification':'category'})

df_cityindex.sort_values('PSGC', inplace = True)

df_cityindex.set_index('PSGC', inplace = True)

df_cityindex.sort_index(inplace = True)

In [24]:
df_cityindex.info()

<class 'pandas.core.frame.DataFrame'>
CategoricalIndex: 1648 entries, 012801000 to 175917000
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Name                   1648 non-null   category
 1   Geographic Level       1648 non-null   category
 2   City Class             1648 non-null   category
 3   Income Classification  1648 non-null   category
 4   Population             1648 non-null   int64   
dtypes: category(4), int64(1)
memory usage: 169.0 KB


In [25]:
df_cityindex.head()

,Name,Geographic Level,City Class,Income Classification,Population
PSGC,,,,,
012801000,ADAMS,Mun,UNK,5th,1792
012802000,BACARRA,Mun,UNK,3rd,32215
012803000,BADOC,Mun,UNK,3rd,31616
012804000,BANGUI,Mun,UNK,4th,14672
012805000,CITY OF BATAC,City,CC,5th,55201


### Cities Geodata

In [26]:
#geojson_path = 'https://github.com/faeldon/philippines-json-maps/tree/master/geojson/municties/medres'
#geojson_path = 'https://github.com/justinelliotmeyers/official_philippines_shapefile_data_2016'
#geojson_path = 'https://raw.githubusercontent.com/macoymejia/geojsonph/master/MuniCities/MuniCities.json'
geojson_path = 'https://raw.githubusercontent.com/macoymejia/geojsonph/master/MuniCities/MuniCities.minimal.json'
#geojson_path = '/Users/gio/Google Drive/Education/IBM Data Science Professional Certificate/Capstone/Coursera_Capstone/municities.json'
df1 = gpd.read_file(geojson_path)
df1.head()

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,NL_NAME_2,VARNAME_2,TYPE_2,ENGTYPE_2,PROVINCE,REGION,geometry
0,177,PHL,Philippines,1,Abra,20,Sallapadan,,,Bayan|Munisipyo,Municipality,Abra,Cordillera Administrative Region (CAR),"POLYGON ((120.78956 17.41699, 120.76131 17.416..."
1,177,PHL,Philippines,1,Abra,21,San Isidro,,,Bayan|Munisipyo,Municipality,Abra,Cordillera Administrative Region (CAR),"POLYGON ((120.63078 17.43194, 120.57896 17.441..."
2,177,PHL,Philippines,1,Abra,22,San Juan,,,Bayan|Munisipyo,Municipality,Abra,Cordillera Administrative Region (CAR),"POLYGON ((120.78275 17.71497, 120.77975 17.665..."
3,177,PHL,Philippines,1,Abra,23,San Quintin,,,Bayan|Munisipyo,Municipality,Abra,Cordillera Administrative Region (CAR),"POLYGON ((120.58414 17.48283, 120.58454 17.476..."
4,177,PHL,Philippines,1,Abra,24,Tayum,,,Bayan|Munisipyo,Municipality,Abra,Cordillera Administrative Region (CAR),"POLYGON ((120.71574 17.62419, 120.72351 17.623..."


In [27]:
df1['Centroid'] = df1.geometry.centroid

In [28]:
df1.Centroid.head()

0    POINT (120.79004 17.48027)
1    POINT (120.61827 17.47013)
2    POINT (120.75506 17.70716)
3    POINT (120.51443 17.51870)
4    POINT (120.67967 17.60724)
Name: Centroid, dtype: geometry

In [30]:
manila_loc = gpd.tools.geocode('Manila')
manila_loc.geometry

0    POINT (120.97480 14.58226)
Name: geometry, dtype: geometry

In [31]:
type(manila_loc.iloc[0,0])

shapely.geometry.point.Point

In [32]:
df1['cent_gs'] = gpd.GeoSeries(df1.Centroid)

In [33]:
df1 = df1.assign(Manila = manila_loc.iloc[0,0])

In [34]:
df1['dist_mnl'] = gpd.GeoSeries(df1.Centroid).distance(manila_loc.iloc[0,0])

In [35]:
df1.sort_values(by = 'dist_mnl')

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,NL_NAME_2,VARNAME_2,TYPE_2,ENGTYPE_2,PROVINCE,REGION,geometry,Centroid,cent_gs,Manila,dist_mnl
995,177,PHL,Philippines,47,Metropolitan Manila,955,Manila,,"City of Manila|Manila, City of",Lungsod|Siyudad,City,Metropolitan Manila,Metropolitan Manila,"POLYGON ((120.98952 14.63567, 120.98637 14.625...",POINT (120.98570 14.59947),POINT (120.98570 14.59947),POINT (120.974800109198 14.5822591781238),0.020368
1000,177,PHL,Philippines,47,Metropolitan Manila,960,Pasay City,,,Lungsod|Siyudad,City,Metropolitan Manila,Metropolitan Manila,"POLYGON ((121.00084 14.50921, 120.99825 14.535...",POINT (121.00359 14.53193),POINT (121.00359 14.53193),POINT (120.974800109198 14.5822591781238),0.057981
1004,177,PHL,Philippines,47,Metropolitan Manila,964,San Juan,,,Lungsod|Siyudad,City,Metropolitan Manila,Metropolitan Manila,"POLYGON ((121.05712 14.60199, 121.03236 14.591...",POINT (121.03392 14.60252),POINT (121.03392 14.60252),POINT (120.974800109198 14.5822591781238),0.062499
994,177,PHL,Philippines,47,Metropolitan Manila,954,Mandaluyong,,"Mandaluyong, City of",Lungsod|Siyudad,City,Metropolitan Manila,Metropolitan Manila,"POLYGON ((121.05753 14.58912, 121.05501 14.574...",POINT (121.03895 14.58382),POINT (121.03895 14.58382),POINT (120.974800109198 14.5822591781238),0.064168
992,177,PHL,Philippines,47,Metropolitan Manila,952,Makati City,,"Makati, City of",Lungsod|Siyudad,City,Metropolitan Manila,Metropolitan Manila,"POLYGON ((121.04446 14.56964, 121.06603 14.560...",POINT (121.03380 14.54904),POINT (121.03380 14.54904),POINT (120.974800109198 14.5822591781238),0.067708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,177,PHL,Philippines,40,Laguna,765,Kalibato Lake,,,Waterbody,Waterbody,Laguna,CALABARZON (Region IV-A),None,None,None,POINT (120.974800109198 14.5822591781238),NaN
818,177,PHL,Philippines,40,Laguna,778,Palakpakin Lake,,,Waterbody,Waterbody,Laguna,CALABARZON (Region IV-A),None,None,None,POINT (120.974800109198 14.5822591781238),NaN
822,177,PHL,Philippines,40,Laguna,782,Sampaloc Lake,,,Waterbody,Waterbody,Laguna,CALABARZON (Region IV-A),None,None,None,POINT (120.974800109198 14.5822591781238),NaN
830,177,PHL,Philippines,40,Laguna,790,Waterbody,,,Waterbody,Waterbody,Laguna,CALABARZON (Region IV-A),None,None,None,POINT (120.974800109198 14.5822591781238),NaN


In [ ]:
df1.Centroid.GeoSeries.distance(manila_loc.geometry)

### Merge data

In [ ]:
df_merged = df_cases_by_city

In [ ]:
df_merged = df_merged.merge(df_cityindex, how = 'left', left_index=True, right_index=True)

In [ ]:
df_merged = df_merged.merge(df_poverty, how = 'left', left_index=True, right_index=True)

In [ ]:
df_merged

#### calculate case/pop as predicted variable
calculate distance from manila as additional feature

In [ ]:
df_merged['Case Per Capita'] = df_merged.Cases/df_merged.Population

In [ ]:
df_merged.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

In [ ]:
df_merged.shape

### 

### Exploratory Data Analysis
So now we have a dataset of cities and municipalities with feature statistics like city class, income classification, population and poverty incidences as possible predictors of cases per capita.

In [ ]:
df_merged.sort_values('Case Per Capita', ascending = False)

In [ ]:
df_merged.info()

Since we have explicitly changed NaNs to literal UNK categorical values, it would be worth checking how many UNKs are there.

In [ ]:
#change UNK for different columns
for columns in df_merged.columns:
    print(f'Percent UNK Column {columns} = {((sum(df_merged[columns] == "UNK"))/len(df_merged[columns])):.0%}')

This points to feature City Class to have mostly UNKs so let's get that off our list of features.

In [ ]:
df_merged = df_merged[['Geographic Level','Income Classification',
           'Population','Poverty Incidence','Case Per Capita']]

In [ ]:
df_merged.tail()

1. Doing pairplots and correlattion analysis shows that of the features mentioned above, none is a strong predictor.

In [ ]:
g = sns.pairplot(df_merged)
g.fig.suptitle("Pair Plot", y=1.08)

In [ ]:
sns.heatmap(df_merged.corr(),annot=True,lw=1)

In [ ]:
df_merged.corr()

2. Separating the analysis by geographic level shows correlation to be stronger at City level.

Let us now explore our data if we group them according to Geographic Level and Income Classification.

First let's do a quick pairplot correlation analysis while differentiating the colors by Geographic Level.

In [ ]:
sns.pairplot(df_merged, hue = "Geographic Level")

In [ ]:
geo_levels = df_merged['Geographic Level'].unique()

In [ ]:
for level in geo_levels:
    sub_df = df_merged.loc[df_merged['Geographic Level'] == level,['Population', 'Poverty Incidence','Case Per Capita']]
    print(level)
    #print('\n')
    print(sub_df_cases_raw.corr())
    print('\n\n')
    g = sns.pairplot(sub_df)
    g.fig.suptitle(level, y=1.08)

We can also see no strong correlation. Relatively, the strongest correlation we see is with Poverty Incidence at -0.52, a moderately negative correlation between Poverty Incidence and Case Per Capita.

Examining the effects if we group them by Income Classification.

In [ ]:
sns.pairplot(df_merged, hue = "Income Classification")

In [ ]:
income_classes = df_merged['Income Classification'].unique()

for level in income_classes:
    sub_df = df_merged.loc[df_merged['Income Classification'] == level,['Population', 'Poverty Incidence','Case Per Capita']]
    print(level)
    #print('\n')
    print(sub_df_cases_raw.corr())
    print('\n\n')
    g = sns.pairplot(sub_df)
    g.fig.suptitle(level, y=1.08)

We can also see no strong correlation. Although only moderate, we do see a moderate positive correlation between Population and Cases per Capita when you're a 6th Class City at 0.64. We also see a moderate negative correlation between Poverty Incidence and Cases per Capita when you are an unclassified city or municipality.

Since most the of cases occur in Metro Manila, let us see if there's Poverty Incidence is a strong predictor of Cases Per Capita in the Metro Manila Area.

We know that cities in the National Capital Region of Metro Manila has a PSGC of 130000000 or more. Limiting our dataset to Metro Manila cities we have:

In [ ]:
#psgc_mm = ['133900000', '137401000', '137402000', '137403000', '137404000', '137405000', '137501000', '137502000', '137503000', '137504000', '137601000', '137602000', '137603000', '137604000', '137606000', '137605000', '137607000']
psgc_mm = ['137401000', '137402000', '137403000', '137404000', '137405000', '137501000', '137502000', '137503000', '137504000', '137601000', '137602000', '137603000', '137604000', '137606000', '137605000', '137607000']
df_mm = df_merged.loc[psgc_mm]

In [ ]:
sns.pairplot(df_mm)

In [ ]:
sns.heatmap(df_mm.corr(),annot=True,lw=1)

In [ ]:
df_mm.corr()

We can also see no strong correlation. We also see a moderate negative correlation between Population and Cases per Capita for the largest cities in the Philippines and the epicenter of the pandemic.

For now, we can conclude that Poverty Incidence is not a strong predictor of severity as measured by Cases Per Capita.

In [ ]:
#dummy variables

#change UNK for different columns
for columns in df_merged.columns:
    print(f'Number of Unknowns for Column {columns} = {sum(df_merged[columns] == "UNK")}')

In [ ]:
df_merged = pd.get_dummies(df_merged, 
               columns = ['Geographic Level', 'Income Classification'], 
               prefix = ['GL','IC'])

In [ ]:
df_merged = df_merged[['Name', 'Population', 'Poverty Incidence', 'GL_City', 
     'GL_Mun', 'GL_SubMun', 'IC_1st', 'IC_2nd','IC_3rd', 'IC_4th', 
     'IC_5th', 'IC_6th', 'IC_Special', 'IC_UNK','Case Per Capita']]

In [ ]:
#check histogram of population
df_merged['Population'][df_merged['IC_3rd'] == 1].hist()

In [ ]:
#normalize population\
ss = StandardScaler()
df_merged['Population Normalized'] = ss.fit_transform(df_merged[['Population']])

In [ ]:
df_merged.corr().loc['Case Per Capita']

In [ ]:
df_merged.head()

### Inference

In [ ]:
#define target variable
y = df_merged['Case Per Capita']

In [ ]:
y

In [ ]:
X = df_merged[['Population Normalized', 'Poverty Incidence', 'GL_City', 'GL_Mun', 'GL_SubMun', 'IC_1st', 'IC_2nd','IC_3rd', 'IC_4th', 'IC_5th', 'IC_6th', 'IC_Special', 'IC_UNK']]

In [ ]:
X

In [ ]:
#split into training and test sets
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

In [ ]:
#fitting a linear model
lm = LinearRegression()
lm.fit(X_train,y_train)

In [ ]:
lm.score(X_train,y_train)

In [ ]:
lm.score(X_valid,y_valid)

In [ ]:
y_pred = lm.predict(X_valid)
rmse = np.sqrt(metrics.mean_squared_error(y_pred, y_valid))
rmse

In [ ]:
rdgCV = RidgeCV(alphas=[0.01,0.1,1,10,100,1000], cv=5)
rdgCV.fit(X_train,y_train)

In [ ]:
print(rdgCV.alpha_)

In [ ]:
rdg = Ridge(alpha=10)
rdg.fit(X_train, y_train)
rdg.score(X_valid, y_valid)

In [ ]:
y_pred = rdg.predict(X_valid)
rmse = np.sqrt(metrics.mean_squared_error(y_pred, y_valid))
rmse

In [ ]:
rfr = RandomForestRegressor(n_jobs=-1, n_estimators=100)
rfr.fit(X,y)

In [ ]:
rfr.score(X_valid,y_valid)

In [ ]:
y_pred = rfr.predict(X_valid)
rmse = np.sqrt(metrics.mean_squared_error(y_pred, y_valid))
rmse

In [ ]:
print(lm.coef_)
print(np.argmax(lm.coef_))
print(df_merged.columns[np.argmax(lm.coef_)])
print(rdgCV.coef_)
print(np.argmax(rdgCV.coef_))

In [ ]:
rfr.fit(X_train,y_train)

y_lm_pred = lm.predict(X_train)
y_rdgCV_pred = rdgCV.predict(X_train)
y_rfr_pred = rfr.predict(X_train)

print('-----training score ---')
print(lm.score(X_train, y_train))
print(rdgCV.score(X_train, y_train))
print(rfr.score(X_train, y_train))
print('----Validation score ---')
print(lm.score(X_valid, y_valid))
print(rdgCV.score(X_valid, y_valid))
print(rfr.score(X_valid, y_valid))

In [ ]:
sns.heatmap(df_merged.corr().loc['Case Per Capita'], robust = True)

# Results
section where you discuss the results.

# Discussion
section where you discuss any observations you noted and any recommendations you can make based on the results.

# Conclusion
section where you conclude the report.

# Extras below
## Next Steps
* merge geojson in main dataframe
* calculate distance from manila
* determine correlation between distance from manila
* do same analysis as earlier with baranggay level

In [ ]:
feature_to_remove = 'Geographic Level'

In [ ]:
sub_df = df_merged

In [ ]:
col_list_new = list(sub_df_cases_raw.columns)
col_list_new

In [ ]:
sns.pairplot(df_merged[['Geographic Level','Population','Poverty Incidence','Case Per Capita']], hue = 'Geographic Level',
            diag_kind="hist")

In [ ]:
def corr_by_level(dataframe, variable):
    for level in geo_levels:
        sub_df = df_merged.loc[df_merged['Geographic Level'] == level,['Population', 'Poverty Incidence','Case Per Capita']]
    print(level)
    print('\n')
    #print(sub_df_cases_raw.corr().loc['Case Per Capita'])
    sns.pairplot(sub_df)
    print('\n\n')

In [ ]:
for level in geo_levels:
    sub_df = df_merged.loc[df_merged['Geographic Level'] == level,['Population', 'Poverty Incidence','Case Per Capita']]
    print(level)
    print('\n')
    #print(sub_df_cases_raw.corr().loc['Case Per Capita'])
    sns.pairplot(sub_df)
    print('\n\n')

In [ ]:
df_merged.columns

In [ ]:
df_merged['Geographic Level'].unique()

In [ ]:
df_merged.loc[df_merged['Geographic Level'] == 'Mun',['Population', 'Poverty Incidence']]

In [ ]:
for level in df_merged['Geographic Level'].unique():
    sub_df = df_merged.loc[df_merged['Geographic Level'] == level,['Population', 'Poverty Incidence','Case Per Capita']]
    print(level)
    print('\n')
    #print(sub_df_cases_raw.corr().loc['Case Per Capita'])
    sns.pairplot(sub_df)
    print('\n\n')

In [ ]:
df_merged['Income Classification'].unique()